# Partitioning and performance

NOTE: THIS NOTEBOOK IS WORK IN PROGRESS.

As Iceberg tables grow to millions of files and trillions of rows, efficiently locating relevant data becomes critical. **Partitioning** is Iceberg's primary tool for this.

In this notebook:

* Why partitioning matters for large tables
* Iceberg's hidden partitioning vs. Hive-style partitioning
* Partition transforms: identity, bucket, truncate, and time-based
* Creating and querying partitioned tables
* Partition evolution: changing partition scheme without rewrites
* IoT-specific partitioning patterns

Partitioning allows Iceberg to skip entire partitions during query execution. For a table with 1 million files across 365 daily partitions, a single-day query needs to scan only ~2,700 files instead of all 1 million.

In [ ]:
import os
import shutil
import daft
import pyarrow as pa
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog
from pyiceberg.transforms import (
    IdentityTransform,
    BucketTransform,
    TruncateTransform,
    DayTransform,
    HourTransform,
    MonthTransform,
    YearTransform,
)

os.environ["DAFT_DASHBOARD_ENABLED"] = "0"
os.environ["DAFT_PROGRESS_BAR"] = "0"

warehouse_path = Path('../data/warehouse_partitioning').absolute()
shutil.rmtree(warehouse_path, ignore_errors=True)
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'
catalog_db.unlink(missing_ok=True)

catalog = SqlCatalog(
    'partitioning_demo',
    **{'uri': f'sqlite:///{catalog_db}', 'warehouse': f'file://{warehouse_path}'}
)
catalog.create_namespace('demo')

# Load events data
df_events = daft.read_json('../data/input/events.jsonl')
print(f"Loaded {df_events.count_rows():,} events")
print("\nInferred schema:")
print(df_events.schema())

## Why partition?

Without partitioning, every query must check every file. With 10,000 Parquet files:

1. Read all 10,000 manifest entries
2. Check per-file statistics (min/max bounds) for each file
3. Read data files that pass the filter

With partitioning by event type (say 50 types), a query for one type:

1. Read manifest entries - and immediately **skip 49 out of 50 partitions** without checking file-level statistics
2. Read only files in the matching partition

This can be a 50x reduction in files to scan.

### Hive-style partitioning vs. hidden partitioning

Traditional **Hive-style partitioning** uses physical directories:

```
events/
  year=2024/month=01/day=01/data.parquet
  year=2024/month=01/day=02/data.parquet
```

Problems:
* Users **must** include partition columns in queries: `WHERE year=2024 AND month=1`
* Changing the partition scheme requires rewriting all data
* Partition columns are duplicate data (stored in both the path and the file)

Iceberg uses **hidden partitioning**:

* Users write normal predicates: `WHERE time > '2024-01-01'`
* Iceberg **automatically maps** the predicate to partition boundaries
* Changing the partition scheme doesn't break existing queries
* Partition columns are **not** stored in the data - only the transform result

The partition structure is "hidden" because users don't need to know about it to write correct queries.

## Creating a partitioned table

Let's create a table partitioned by event `type` using the `IdentityTransform`. This means each unique value of `type` gets its own partition.

After creating the table, we use `update_spec()` to define the partition. This allows us to add partitioning to an existing table schema, and also to **evolve** the partition scheme later without rewriting data.

In [ ]:
# Create table from inferred schema
arrow_sample = df_events.limit(1000).to_arrow()
events_table = catalog.create_table(
    'demo.events_by_type',
    schema=pa.schema(arrow_sample.schema)
)

# Add partition spec: partition by 'type' using identity transform
# IdentityTransform means: one partition per unique value of 'type'
with events_table.update_spec() as update:
    update.add_field(
        source_column_name='type',
        transform=IdentityTransform(),
        partition_field_name='type'
    )

print("✅ Created partitioned table")
print(f"\nPartition spec:")
for field in events_table.spec().fields:
    print(f"  Field '{field.name}': {field.transform} on source field ID {field.source_id}")

# Write a larger sample
arrow_batch = df_events.limit(50000).to_arrow()
events_table.append(arrow_batch)
print(f"\n✅ Appended {len(arrow_batch):,} records")

### File structure with partitioning

Let's look at how the data files are organized. Each partition gets its own directory named after the partition value.

In [ ]:
# Show the partition directory structure
table_dir = Path(events_table.location().replace('file://', ''))
data_dir = table_dir / 'data'

partitions = sorted(data_dir.iterdir()) if data_dir.exists() else []

print(f"Data directory: {data_dir.relative_to(warehouse_path)}")
print(f"Number of partitions: {len(partitions)}")
print()

total_size = 0
for partition_dir in sorted(partitions):
    files = list(partition_dir.glob('*.parquet'))
    partition_size = sum(f.stat().st_size for f in files)
    total_size += partition_size
    print(f"  {partition_dir.name}/")
    for f in files:
        print(f"    {f.name} ({f.stat().st_size:,} bytes)")

print(f"\nTotal data size: {total_size / 1024:.1f} KB")

## Partition pruning in queries

When Daft queries an Iceberg table with a filter on the partition column, it can skip entire partitions. The manifest list already knows which event types are in which manifests, so Daft never needs to read irrelevant partitions.

Note that this works even though our query uses the regular `type` column name - Daft knows internally that `type` is a partition column and applies partition pruning automatically.

In [ ]:
# First: check what event types exist
df = daft.read_iceberg(events_table)
print("Event types in the table:")
daft.sql("""
    SELECT type, COUNT(*) as count
    FROM df
    GROUP BY type
    ORDER BY count DESC
""").show()

In [ ]:
# Query for a specific type - Daft will only read the matching partition
df_filtered = daft.read_iceberg(events_table)
df_alarm = df_filtered.filter(daft.col('type') == 'c8y_LocationUpdate')
print("Querying for type = 'c8y_LocationUpdate'")
print("(Only the matching partition directory will be read)")
print()
df_alarm.select('id', 'type', 'time', 'source').show(5)

In [ ]:
# Inspect the manifest to see how partition information is stored
import fastavro
from pathlib import Path

# Find a manifest file
metadata_dir = table_dir / 'metadata'
manifest_files = list(metadata_dir.glob('*-m*.avro'))

if manifest_files:
    manifest = manifest_files[0]
    with open(manifest, 'rb') as f:
        reader = fastavro.reader(f)
        records = list(reader)

    print(f"Manifest: {manifest.name}")
    print(f"Contains {len(records)} file entries\n")

    # Show partition info for first few entries
    for i, record in enumerate(records[:5], 1):
        data_file = record.get('data_file', {})
        partition = data_file.get('partition', {})
        print(f"  Entry {i}: partition={partition}, records={data_file.get('record_count', 0):,}")

## Partition transforms

Iceberg supports several transforms that control how partition values are computed from column values:

| Transform | Input type | Output | Use case |
|-----------|-----------|--------|----------|
| `identity` | Any | Column value unchanged | Exact value partitioning for low cardinality |
| `year` | timestamp/date | Year integer | Annual partitioning |
| `month` | timestamp/date | Year+month integer | Monthly partitioning |
| `day` | timestamp/date | Date integer | Daily partitioning |
| `hour` | timestamp | Year+hour integer | Hourly partitioning |
| `bucket(n)` | int, long, string, date, timestamp | Bucket 0..n-1 | High-cardinality columns |
| `truncate(w)` | int, long, string | First w chars (string) or rounded (int/long) | Prefix partitioning |

### Choosing the right transform

* **Identity**: Use when cardinality is low (< 100 distinct values) and values are stable. Good for: event type, status, region.
* **Time-based (year/month/day/hour)**: Use for time-series data. Choose granularity based on query patterns and file sizes. For IoT with millions of events/day: `hour`. For slower data: `day` or `month`.
* **Bucket**: Use for high-cardinality ID columns (device ID, source ID). Distributes data evenly, avoids creating too many partitions. Good for: device_id, tenant_id, user_id.
* **Truncate**: Use for string prefixes or integer ranges. Less common.

### Time-based partitioning

For IoT time-series data, partitioning by time is the most common pattern. Iceberg transforms timestamps into year, month, day, or hour partitions.

> **Note:** The time-based transforms (`DayTransform`, `HourTransform`, etc.) require the source column to be of a **timestamp** type in the Iceberg schema. If your time column is stored as a string, you need to cast it to a timestamp type first.

Let's create a time-partitioned table:

In [ ]:
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, TimestampType

# Create a synthetic table with an explicit timestamp column to demonstrate time partitioning
# We use PyArrow's timestamp type so Iceberg gets a proper timestamp field
import pyarrow as pa
import numpy as np
from datetime import datetime, timedelta

# Generate synthetic time-series data spanning multiple days
np.random.seed(42)
n_records = 10000
base_time = datetime(2024, 1, 1)
timestamps = [base_time + timedelta(hours=i * 0.5) for i in range(n_records)]

ts_data = pa.table({
    'event_time': pa.array(timestamps, type=pa.timestamp('us')),
    'device_id': pa.array([f'device_{i % 100}' for i in range(n_records)]),
    'value': pa.array(np.random.randn(n_records).tolist()),
    'event_type': pa.array(['alarm' if i % 10 == 0 else 'measurement' for i in range(n_records)])
})

print(f"Generated {n_records:,} synthetic IoT records")
print(f"Time range: {timestamps[0].date()} to {timestamps[-1].date()}")
print(f"Schema: {ts_data.schema}")

In [ ]:
# Create table and partition by day
ts_table = catalog.create_table(
    'demo.timeseries',
    schema=pa.schema(ts_data.schema)
)

# Add day-based partition on the timestamp column
with ts_table.update_spec() as update:
    update.add_field(
        source_column_name='event_time',
        transform=DayTransform(),
        partition_field_name='event_day'
    )

print("✅ Created time-partitioned table")
print(f"\nPartition spec: {ts_table.spec()}")

# Write data
ts_table.append(ts_data)
print(f"\n✅ Appended {n_records:,} records")

# Show resulting partitions
ts_dir = Path(ts_table.location().replace('file://', ''))
ts_data_dir = ts_dir / 'data'

print("\nResulting partitions (by day):")
if ts_data_dir.exists():
    for partition_dir in sorted(ts_data_dir.iterdir()):
        files = list(partition_dir.glob('*.parquet'))
        total = sum(f.stat().st_size for f in files)
        print(f"  {partition_dir.name} → {len(files)} file(s), {total:,} bytes")

In [ ]:
# Query with a time filter - only matching day partitions will be read
df_ts = daft.read_iceberg(ts_table)

print("Counting records per day:")
daft.sql("""
    SELECT
        CAST(event_time AS DATE) as day,
        COUNT(*) as event_count,
        SUM(CASE WHEN event_type = 'alarm' THEN 1 ELSE 0 END) as alarms
    FROM df_ts
    GROUP BY CAST(event_time AS DATE)
    ORDER BY day
    LIMIT 10
""").show()

### Bucket partitioning

**Bucket** partitioning is designed for high-cardinality columns like device IDs or user IDs. Instead of creating one partition per unique value (which could be millions), it hashes values into a fixed number of buckets.

With `BucketTransform(n=16)`, Iceberg applies a hash function and takes the result modulo 16. This guarantees exactly 16 partitions regardless of how many distinct device IDs exist.

Benefits:
* **Controlled partition count**: Always exactly n partitions
* **Even distribution**: Hash functions distribute data uniformly
* **Good for joins**: Two tables with the same bucket spec on the same column can be joined without shuffling

The trade-off: You can no longer filter by a specific device ID at the partition level (the query engine must check all buckets). But you gain control over partition count.

In [ ]:
# Create a table partitioned by bucket on device_id
bucket_table = catalog.create_table(
    'demo.timeseries_bucketed',
    schema=pa.schema(ts_data.schema)
)

NUM_BUCKETS = 8

with bucket_table.update_spec() as update:
    update.add_field(
        source_column_name='device_id',
        transform=BucketTransform(num_buckets=NUM_BUCKETS),
        partition_field_name='device_bucket'
    )

bucket_table.append(ts_data)
print(f"✅ Wrote {n_records:,} records to {NUM_BUCKETS}-bucket partitioned table")

# Show bucket distribution
bucket_dir = Path(bucket_table.location().replace('file://', '')) / 'data'

print("\nBucket distribution:")
if bucket_dir.exists():
    for partition_dir in sorted(bucket_dir.iterdir()):
        files = list(partition_dir.glob('*.parquet'))
        total = sum(f.stat().st_size for f in files)
        print(f"  {partition_dir.name}: {total:,} bytes")

print(f"\nWith {n_records:,} records across {n_records // 100} unique devices and {NUM_BUCKETS} buckets,")
print(f"each bucket contains roughly {n_records // NUM_BUCKETS:,} records.")

### Multi-column partitioning

You can combine multiple partition transforms to create a hierarchical partition scheme. For IoT data, a natural combination is:

* **Day** on time: separate data by date
* **Bucket** on device_id: within each day, distribute devices across buckets

This creates a two-level hierarchy. A query like `WHERE event_time = '2024-01-15' AND device_id = 'device_42'` can skip:

1. All partitions not in day `2024-01-15`
2. Within that day, all buckets that don't contain `device_42`'s hash

In [ ]:
# Create table with combined day + bucket partitioning
multi_table = catalog.create_table(
    'demo.timeseries_multi',
    schema=pa.schema(ts_data.schema)
)

with multi_table.update_spec() as update:
    update.add_field(
        source_column_name='event_time',
        transform=DayTransform(),
        partition_field_name='event_day'
    )
    update.add_field(
        source_column_name='device_id',
        transform=BucketTransform(num_buckets=4),
        partition_field_name='device_bucket'
    )

multi_table.append(ts_data)

print("✅ Created multi-column partitioned table")
print(f"\nPartition spec:")
for field in multi_table.spec().fields:
    print(f"  {field.name}: {field.transform}")

# Show the resulting structure
multi_dir = Path(multi_table.location().replace('file://', '')) / 'data'
print("\nPartition directories (day/bucket):")
if multi_dir.exists():
    partitions = sorted(multi_dir.iterdir())
    shown = 0
    for p in partitions:
        sub_partitions = sorted(p.iterdir()) if p.is_dir() else []
        for sp in sub_partitions[:2]:  # Show first 2 sub-partitions per day
            files = list(sp.glob('*.parquet'))
            print(f"  {p.name}/{sp.name}/: {len(files)} file(s)")
            shown += 1
        if len(sub_partitions) > 2:
            print(f"  {p.name}/... and {len(sub_partitions) - 2} more bucket(s)")
        if shown >= 10:
            remaining = sum(1 for _ in multi_dir.iterdir()) - shown
            if remaining > 0:
                print(f"  ... and more partitions")
            break

## Partition evolution

One of Iceberg's most powerful features: you can **change the partition scheme without rewriting data**.

Suppose you initially partition by month but queries become too slow because monthly partitions are too large. You want to switch to daily partitioning. In Hive, this would require rewriting terabytes of data. In Iceberg:

1. Add the new partition spec
2. New writes use the new spec
3. Old data remains in old partitions
4. Queries work transparently across both old and new partition schemes

This is possible because Iceberg stores the partition spec **per data file** in the manifest. The query engine knows which spec each file uses and applies the appropriate pruning.

Let's demonstrate: start with monthly partitioning, then evolve to daily.

In [ ]:
# Create table with monthly partitioning first
evolving_table = catalog.create_table(
    'demo.timeseries_evolving',
    schema=pa.schema(ts_data.schema)
)

with evolving_table.update_spec() as update:
    update.add_field(
        source_column_name='event_time',
        transform=MonthTransform(),
        partition_field_name='event_month'
    )

# Write first batch with monthly partitioning
evolving_table.append(ts_data)

print("Phase 1: Monthly partitioning")
print(f"Spec ID: {evolving_table.spec().spec_id}")
evol_dir = Path(evolving_table.location().replace('file://', '')) / 'data'
if evol_dir.exists():
    for p in sorted(evol_dir.iterdir()):
        print(f"  {p.name}")

In [ ]:
# Evolve to daily partitioning - no data rewrite needed!
with evolving_table.update_spec() as update:
    update.remove_field('event_month')  # Remove old partition
    update.add_field(
        source_column_name='event_time',
        transform=DayTransform(),
        partition_field_name='event_day'
    )

# Reload to get the updated spec
evolving_table = catalog.load_table('demo.timeseries_evolving')

print("Phase 2: Evolved to daily partitioning")
print(f"New spec ID: {evolving_table.spec().spec_id}")

# Write more data with the new spec
# Generate data for the next month
base_time2 = datetime(2024, 3, 1)
timestamps2 = [base_time2 + timedelta(hours=i * 0.5) for i in range(5000)]
ts_data2 = pa.table({
    'event_time': pa.array(timestamps2, type=pa.timestamp('us')),
    'device_id': pa.array([f'device_{i % 100}' for i in range(5000)]),
    'value': pa.array(np.random.randn(5000).tolist()),
    'event_type': pa.array(['alarm' if i % 10 == 0 else 'measurement' for i in range(5000)])
})

evolving_table.append(ts_data2)

print("\nData directory after evolution:")
if evol_dir.exists():
    for p in sorted(evol_dir.iterdir()):
        if p.is_dir():
            sub_dirs = list(p.iterdir()) if p.name.startswith('event_month') else []
            if sub_dirs:
                print(f"  {p.name}/ (old monthly partition)")
            else:
                print(f"  {p.name}/ (new daily partition)")

print("\n✅ Old data stays in monthly partitions, new data uses daily partitions")
print("   Queries work transparently across both!")

In [ ]:
# Queries work transparently across old and new partition schemes
df_evolving = daft.read_iceberg(evolving_table)

print("Total records across all partition specs:")
daft.sql("SELECT COUNT(*) as total FROM df_evolving").show()

print("\nRecords by date (spans both partition schemes):")
# Cast to date and count records per date
df_by_date = daft.sql("""
    SELECT
        CAST(event_time AS DATE) as event_date,
        COUNT(*) as count
    FROM df_evolving
    GROUP BY CAST(event_time AS DATE)
    ORDER BY event_date
""")
df_by_date.show(10)

## IoT partitioning patterns

For IoT data, the right partitioning strategy depends on your query patterns:

### Pattern 1: Time-first (most common)
```
Partition by: day(event_time), bucket(device_id, 16)
```
Best for:
* Most queries filter by time range
* Dashboards showing recent data
* Retention policies (delete old day partitions)

### Pattern 2: Device-first
```
Partition by: bucket(device_id, 64), day(event_time)
```
Best for:
* Device-specific analytics dominate
* Queries like "all events for device X in the last year"

### Pattern 3: Type-based routing
```
Partition by: identity(event_type), day(event_time)
```
Best for:
* Few distinct event types
* Analytics segregated by type (alarms vs. measurements)
* Type-specific retention policies

### Key guidelines

* **Avoid identity partitioning on high-cardinality columns**: If you have 10 million device IDs, you get 10 million partitions → metadata overhead becomes extreme
* **Target file size**: Aim for 128 MB-1 GB per Parquet file. Too many small files hurt performance.
* **Partition granularity**: With 1 million events/day, use `day`. With 100 events/day, use `month`.
* **Query patterns drive design**: Partition to skip most data for your most frequent queries

## Review questions

**What is the key difference between Hive-style and Iceberg hidden partitioning?**
   - How does each affect the queries users write?
   - What happens when you change the partition scheme?

**Why is identity partitioning dangerous for high-cardinality columns?**
   - What happens to manifest size with 10 million partitions?
   - What would you use instead?

**How does partition evolution work without data rewrites?**
   - Where is the partition spec stored per file?
   - How can old and new partition specs coexist?

**When should you use bucket vs. day partitioning on a timestamp column?**
   - What queries benefit from day partitioning?
   - Can bucket partitioning coexist with day partitioning?

**How do you choose the number of buckets for BucketTransform?**
   - What factors influence this decision?
   - Can you change the number of buckets later?

**What is the "small files problem" and how does partitioning contribute to it?**
   - When does this become a problem?
   - What can you do about it?

## Challenges

### Partition the events table by type and compare performance

1. Create an unpartitioned version of the events table with 100,000 records
2. Create a version partitioned by `identity(type)`
3. Query both tables for a specific event type
4. How many files are read in each case? Check the manifest entries.

### Experiment with bucket counts

1. Create three tables partitioned by `bucket(device_id, n)` with n=4, 8, 16
2. Write the same 10,000 records to all three
3. Check how evenly data is distributed across buckets
4. Which count gives the most even distribution?

### IoT data partitioning analysis

Using the events.jsonl data:
1. Count distinct values of `type` field
2. Decide: is identity or bucket better for `type`?
3. If `time` is a timestamp, what granularity (hour/day/month) makes sense?
   - Hint: how many events are there per day?
4. Design the optimal partition spec for this dataset

### Partition evolution experiment

1. Create a table with `identity(type)` partitioning
2. Write 20,000 events
3. Evolve the partition spec to add `bucket(type, 4)` instead
4. Write another 10,000 events
5. Verify that queries return the correct total across both partition specs

## Summary

Partitioning is one of the most impactful performance features in Iceberg:

* **Hidden partitioning**: Users write normal predicates; Iceberg handles partition pruning automatically
* **Transforms**: Identity, bucket, truncate, and time-based (year/month/day/hour) handle all common patterns
* **Partition pruning**: Entire partitions are skipped during query planning, before reading any data files
* **Partition evolution**: Change the partition scheme without rewriting data - old and new specs coexist transparently
* **Multi-column partitioning**: Combine transforms for hierarchical partition schemes

### Key takeaways

1. **Design for your queries**: Partition to skip data for your most frequent predicates
2. **Avoid high-cardinality identity**: Use bucket for device IDs, user IDs, etc.
3. **Time-first for IoT**: Time-based partitioning is almost always beneficial for time-series data
4. **Partition evolution is safe**: Don't be afraid to change the partition scheme as requirements evolve
5. **Balance granularity**: Too fine → too many small files; too coarse → too little pruning

### What's next?

Next, we'll look at how Iceberg works on object stores like S3 - the standard deployment target for production data lakes.